## 11_1. T -- Cell Annotation

<div 
    <p style="text-align: left;">Updated Time: 2025-06-03</p>
</div>

##### Load libraries

In [ ]:
import os
import sys
import numpy as np
import pandas as pd

import omicverse as ov
import scanpy as sc
import decoupler as dc
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib.pyplot import rc_context
from pyclustree import clustree
ov.plot_set()

import warnings
warnings.simplefilter("ignore")

##### Set working directory for analysis

In [ ]:
cwd = '/media/bio/Disk/Research Data/EBV/omicverse'
os.chdir(cwd)
updated_dir = os.getcwd()
print("Updated working directory: ", updated_dir)

from pathlib import Path
saving_dir = Path('Results/11.TCell')
saving_dir.mkdir(parents=True, exist_ok=True)

##### Reading in annotated AnnData object

In [ ]:
adata = sc.read_h5ad("Processed Data/scRNA_Annotation.h5ad")
adata

In [ ]:
for i in adata.obs['Cell_type'].cat.categories:
  number = len(adata.obs[adata.obs['Cell_type']==i])
  print('the number of category {} is {}'.format(i,number))

In [ ]:
# Select myeloid cells for downstream analysis
adata_tcell = adata[adata.obs['Cell_type'].isin(['T'])].copy()
adata_tcell

In [ ]:
print(np.min(adata_tcell.X), np.max(adata_tcell.X))

In [ ]:
adata_tcell.obs['EBV_status'] = adata_tcell.obs['EBV_status'].cat.reorder_categories(['Normal', 'Negative', 'Positive'])

In [ ]:
for i in adata_tcell.obs['EBV_status'].cat.categories:
  number = len(adata_tcell.obs[adata_tcell.obs['EBV_status']==i])
  print('the number of category {} is {}'.format(i,number))


#### Preprocessing

You can use `recover_counts` to recover the raw counts after normalize and log1p

In [ ]:
X_counts_recovered, size_factors_sub=ov.pp.recover_counts(adata_tcell.X, 50*1e4, 50*1e5, log_base=None, chunk_size=50000)
adata_tcell.layers['counts']=X_counts_recovered

In [ ]:
adata_tcell.X=adata_tcell.layers['counts']
print(np.min(adata_tcell.X), np.max(adata_tcell.X))

In [ ]:
adata_tcell=ov.pp.preprocess(adata_tcell,mode='shiftlog|pearson',n_HVGs=2000,)
adata_tcell.raw = adata_tcell
adata_tcell = adata_tcell[:, adata_tcell.var.highly_variable_features]
ov.pp.scale(adata_tcell)
ov.pp.pca(adata_tcell,layer='scaled',n_pcs=50)

In [ ]:
ov.utils.plot_pca_variance_ratio(adata_tcell)

In [ ]:
ov.single.batch_correction(adata_tcell,batch_key='Dataset', methods='harmony',max_iter_harmony = 20, n_pcs=10) 

#### Unsupervised clustering
The Leiden algorithm is as an improved version of the Louvain algorithm which outperformed other clustering methods for single-cell RNA-seq data analysis ([Du et al., 2018, Freytag et al., 2018, Weber and Robinson, 2016]). Since the Louvain algorithm is no longer maintained, using Leiden instead is preferred.

We, therefore, propose to use the Leiden algorithm[Traag et al., 2019] on single-cell k-nearest-neighbour (KNN) graphs to cluster single-cell datasets.

Leiden creates clusters by taking into account the number of links between cells in a cluster versus the overall expected number of links in the dataset.

Here, we set method='leiden' to cluster the cells using Leiden

In [ ]:
ov.pp.neighbors(adata_tcell, n_neighbors=15, use_rep='X_harmony')

In [ ]:
ov.pp.umap(adata_tcell)

In [ ]:
ov.utils.embedding(adata_tcell,basis='X_umap',
                   color=['EBV_status'],
                   frameon='small',wspace=0.5)

In [ ]:
# Run leiden clustering for different resolutions
for resolution in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]:
    ov.pp.leiden(
        adata_tcell,
        resolution=resolution,
        flavor="igraph",
        n_iterations=2,
        key_added=f"leiden_{str(resolution).replace('.', '_')}",
    )

#### Plot the clustree

In [ ]:
# Plot the clustree
fig = clustree(
    adata_tcell,
    [f"leiden_{str(resolution).replace('.', '_')}" for resolution in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]],
    title="Clustree of T Cell Clusters",
    edge_weight_threshold=0.00,  # the minimum fraction of the parent cluster assigned to the child cluster to plot
    show_fraction=True,  # show the fraction of cells in each cluster
)
fig.set_size_inches(10, 8)
fig.set_dpi(100)

In [ ]:
for i in adata_tcell.obs['leiden_1_0'].cat.categories:
  number = len(adata_tcell.obs[adata_tcell.obs['leiden_1_0']==i])
  print('the number of category {} is {}'.format(i,number))

Based on the cluster scoring，a resolution of 0.4 may be the optimal. Here we visualize the optimal clustering using UMAP representation:

In [ ]:
from matplotlib import patheffects
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(4,4))

ov.pl.embedding(adata_tcell,
                  basis='X_umap',
                  color=['leiden_1_0'],
                  show=False, legend_loc=None, add_outline=False, 
                  frameon='small',legend_fontoutline=2,ax=ax
                 )

ov.utils.gen_mpl_labels(
    adata_tcell,
    'leiden_1_0',
    exclude=("None",),  
    basis='X_umap',
    ax=ax,
    adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
    text_kwargs=dict(fontsize= 12 ,weight='bold',
                     path_effects=[patheffects.withStroke(linewidth=2, foreground='w')] ),
)

In [ ]:
ov.pl.cellproportion(
    adata=adata_tcell,
    celltype_clusters='leiden_1_0',
    groupby='EBV_status',
    legend=True,
)

#### Finding marker genes

In [ ]:
adata_tcell = adata_tcell.raw.to_adata() # This recovers the raw count data in adata.X

Let us compute a ranking for the highly differential genes in each cluster. For this, by default, the .raw attribute of AnnData is used in case it has been initialized before. The simplest and fastest method to do so is the t-test.

In [ ]:
sc.tl.dendrogram(adata_tcell,'leiden_1_0',use_rep='X_harmony')

plt.rcParams["figure.figsize"] = [10, 4]
sc.pl.dendrogram(adata_tcell,'leiden_1_0')

In [ ]:
sc.tl.rank_genes_groups(adata_tcell, 'leiden_1_0', method='t-test',use_raw=False,key_added='leiden_1_0_ttest')
sc.pl.rank_genes_groups_dotplot(adata_tcell,groupby='leiden_1_0',
                                cmap='Spectral_r',key='leiden_1_0_ttest',
                                standard_scale='var',n_genes=3)

Output the marker list as pandas dataframe:

In [ ]:
ttest_marker_gene=pd.DataFrame(adata_tcell.uns['leiden_1_0_ttest']['names'])
ttest_marker_gene.head()

In [ ]:
ttest_marker_gene=ttest_marker_gene.head(50)
ttest_marker_gene.to_csv('Results/11.TCell/ttest_marker_gene_leiden_1_0_ttest.csv', index=False)

cosg is also considered to be a better algorithm for finding marker genes. Here, omicverse provides the calculation of cosg

Paper: Accurate and fast cell marker gene identification with COSG

Code: https://github.com/genecell/COSG

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='leiden_1_0', method='t-test',key_added='leiden_1_0_ttest', use_rep='scaled|original|X_pca')

adata_tcell.uns['X_sparse_backup'] = adata_tcell.X.copy()

adata_tcell.X = adata_tcell.X.toarray()
ov.single.cosg(adata_tcell, key_added='leiden_1_0_cosg', groupby='leiden_1_0')

adata_tcell.X = adata_tcell.uns['X_sparse_backup']
del adata_tcell.uns['X_sparse_backup']

sc.pl.rank_genes_groups_dotplot(adata,groupby='leiden_1_0',
                                cmap='Spectral_r',key='leiden_1_0_cosg',
                                standard_scale='var',n_genes=3)

Output the marker list as pandas dataframe:

In [ ]:
cosg_marker_gene=pd.DataFrame(adata_tcell.uns['leiden_1_0_cosg']['names'])
cosg_marker_gene.head()

In [ ]:
cosg_marker_gene=cosg_marker_gene.head(100)
cosg_marker_gene.to_csv('Results/11.TCell/cosg_marker_gene_leiden_1_0.csv', index=False)

##### Cell type annotation from marker genes

In [ ]:
from matplotlib import patheffects
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(4,4))

ov.pl.embedding(adata_tcell,
                  basis='X_umap',
                  color=['leiden_1_0'],
                  show=False, legend_loc=None, add_outline=False, 
                  frameon='small',legend_fontoutline=2,ax=ax
                 )

ov.utils.gen_mpl_labels(
    adata_tcell,
    'leiden_1_0',
    exclude=("None",),  
    basis='X_umap',
    ax=ax,
    adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
    text_kwargs=dict(fontsize= 12 ,weight='bold',
                     path_effects=[patheffects.withStroke(linewidth=2, foreground='w')] ),
)

#### DIY a collection of gene sets

In [ ]:
Tcell_Functional_Signatures = pd.read_excel('Dataset/Tcell_Functional_Signatures.xlsx', sheet_name=0)

data = []
for column in Tcell_Functional_Signatures.columns:
    genes = Tcell_Functional_Signatures[column].dropna().astype(str).tolist()
    for gene in genes:
        data.append({'source': column, 'target': gene})

Tcell_Functional_Signatures = pd.DataFrame(data)

Tcell_Functional_Signatures.head()

We can easily compute cell type enrichment scores by running the ulm method.

In [ ]:
net=Tcell_Functional_Signatures.drop_duplicates(subset=['source', 'target'])

In [ ]:
# Runs multiple enrichment methods sequentially.
dc.mt.aucell(data=adata_tcell, net=net, tmin=1)
adata_tcell

In [ ]:
scores = dc.pp.get_obsm(adata_tcell, key='score_aucell')

In [ ]:
sc.pl.matrixplot(
    adata=scores,
    var_names=scores.var_names,
    groupby='leiden_1_0',
    dendrogram=True,
    standard_scale='var',
    colorbar_title='Z-scaled scores',
    cmap='RdBu_r',
)

Based on the literature and existing knowledge, a feature dictionary was constructed by integrating the marker genes of each subpopulation obtained from the previous section, defining potential cell subtypes and their corresponding marker genes.

In [ ]:
tcell_genes_dict1 = {
    'T': ['CD3D', 'CD3E', 'CD3G'], 
    'CD8+': ['CD8A', 'CD8B', 'PRF1', 'IFNG', 'LAMP1'],
    'CD4+': ['CD4',  'IL2RA','CCR7','CXCR5','CD40LG'],
    'Treg': ['FOXP3', 'CTLA4', 'TNFRSF18'],
    'Proliferative': ['TOP2A', 'MKI67', 'STMN1'],
}

sc.pl.dotplot(adata_tcell, tcell_genes_dict1, 'leiden_1_0', dendrogram=False, use_raw=False, standard_scale='var', show=True, save='11_1. Dotplot_TcellCluster_Curated_Markers_leiden_1_0.pdf.pdf')

In [ ]:
# Pan-cancer single-cell landscape of tumor-infiltrating T cells
cd8_genes_dict = ['TCF7', 'LEF1', 'CCR7', 'SELL', 'MAL', 
                  'IL7R', 'GPR183', 'ZFP36L2', 'CXCR4', 
                  'ZNF683', 'CD52', 'HOPX', 'ID2', 'CXCR6', 'XCL1', 'XCL2', 
                  'TBX21', 'ASCL2',  'CX3CR1', 'KLRG1',
                  'KLRD1', 'TYROBP', 'KIR2DL3', 'KIR2DL1', 'KIR3DL1', 'KIR3DL2',  'CD160',  'EOMES',  'TXK',  'KLRC1',  'KIR2DL4', 
                  'GZMK', 'CXCR5', 'CCR4', 'CD28', 'CXCR3', 'GZMH', 'CD27', 'HLA-DRB1', 
                  'PDCD1', 'CXCL13', 'LAYN', 
                  'STAT1', 'IFIT1', 'ISG15', 'CCR1', 
                  'SLC4A10', 'KLRB1', 'TMIGD2', 'RORA', 'RORC', 'ZBTB16', 'IL26', 'IL17A', 'IL23R', 
                  'NME1', 'NME2', 'MND1', 'SPC24', 'MYB']  

cd4_genes_dict = ['CD40LG', 
                  'FOXP3', 
                  'TCF7', 'LEF1', 'TXK', 'CCR7', 'SELL', 'MAL', 'CXCR5', 'ADSL', 'IL16', 'IL7R', 
                  'TNF', 'AREG', 'TIMP1', 'CREM', 'CCL5', 'CAPG', 'GZMK', 'KLRG1', 'CX3CR1', 'TBX21', 
                  'RORA', 'RORC', 'CCR6', 'IL23R', 'IL22', 'IL17A', 'IL17F', 'IL26', 
                  'TOX', 'TOX2', 'IL21', 'CXCL13', 'GNG4', 'CD200', 'BCL6', 'ZBED2', 'CCL3', 'CCL4', 'IFNG', 'GZMB', 'LAG3', 'HAVCR2', 
                  'RTKN2', 'IL2RA', 'S1PR1', 'TNFRSF9', 'CTLA4', 'LAYN', 'STAT1', 'IFIT1', 'IRF7', 
                  'NME1', 'NME2', 'SPC24', 'CCR4', ] # 'MND1'

In [ ]:
sc.pl.dotplot(adata_tcell, cd8_genes_dict, 'leiden_1_0', dendrogram=False, use_raw=False, standard_scale='var', show=True, save='11_1. Dotplot_cd8Cluster_Curated_Markers1_leiden_1_0.pdf.pdf')
sc.pl.dotplot(adata_tcell, cd4_genes_dict, 'leiden_1_0', dendrogram=False, use_raw=False, standard_scale='var', show=True, save='11_1. Dotplot_cd4Cluster_Curated_Markers1_leiden_1_0.pdf.pdf')

In [ ]:
# Pan-cancer T cell atlas links a cellular stress response state to immunotherapy resistance
cd8_genes_dict = ['GZMK', 'GZMB', 'PRF1', 'CD44', 'CD69', 
                  'TOX', 'NR3C1', 'PDCD1', 'LAG3', 'CTLA4', 
                  'FGFBP2', 'GZMH', 'GNLY', 'NR4A1', 'BAG3', 
                  'HSPA1A', 'IFIT1', 'MX1', 'DKK3', 'CCR4', 
                  'EOMES', 'CNN2', 'LIMD2', 'CD27', 'LGR6', 
                  'KLRC4', 'CD244', 'SEMA4A', 'ITGA1', 'KLRB1', 
                  'PRDM1', 'CCR7', 'SELL', 'TCF7', 'TRGV5', 'TRGV10']

cd4_genes_dict = ['IL7R', 'CD69', 'GPR183', 'CTLA4', 'IL2RA', 
                  'FOXP3', 'TNFRSF4', 'RPL31', 'RPL21', 'CXCL13', 
                  'NR4A1', 'BCL6', 'ICOS', 'TOX', 'PDCD1', 
                  'BAG3', 'FOS', 'JUN', 'IFNG', 'GZMA', 
                  'GZMH', 'GZMB', 'GZMK', 'SELL', 'LEF1', 
                  'CCR7', 'FHIT', 'PRF1', 'NKG7', 'TCF7', 
                  'TCEA3', 'CDC25B', 'IL17A', 'IL17F', 'RORA', 
                  'KLRB1', 'CCR6', 'SLC40A1', 'ANKRD55', 'ISG15', 
                  'IFI44L', 'IFIT1']

treg_genes_dict = ['KLF2', 'LEF1', 'SELL', 'IL1R2', 'IL1R1', 
                   'TNFRSF4', 'TNFRSF18', 'TNFRSF9', 'IL2RA', 'BATF', 
                   'IL21R', 'FOXP3', 'FANK1', 'GATA3', 'TNFRSF13B', 
                   'GPX1', 'HSPA1A', 'HSPA1B', 'BAG3', 'HSPE1', 
                   'HSPB1', 'HSPD1', 'NR4A1', 'IL7R', 'LRRC32', 
                   'CCR7', 'ICA1', 'CXCL13', 'TCF7', 'KLRB1']

tfh_genes_dict = ['NMB', 'IL7R', 'CXCL13', 'CD200', 'CCR7', 
                  'PDCD1', 'TOX2', 'SH2D1A', 'IL6R', 'CD40LG', 
                  'TIGIT', 'CXCR5', 'ASCL2', 'ICOS', 'KLRB1', 
                  'GPR183', 'CD69', 'KLF2', 'CCR5', 'TGFB1', 
                  'PRF1', 'PRDM1', 'NKG7', 'MAF', 'LAG3', 
                  'IL2RG', 'IL2RB', 'IL10RA', 'IFNG', 'HAVCR2', 
                  'GZMB', 'GZMA', 'GNLY', 'ENTPD1', 'IRF4', 
                  'BATF', 'TNFRSF4', 'IL2RA', 'NRN1']

In [ ]:
sc.pl.dotplot(adata_tcell, cd8_genes_dict, 'leiden_1_0', dendrogram=False, use_raw=False, standard_scale='var', show=True, save='11_1. Dotplot_cd8Cluster_Curated_Markers2_leiden_1_0.pdf.pdf')
sc.pl.dotplot(adata_tcell, cd4_genes_dict, 'leiden_1_0', dendrogram=False, use_raw=False, standard_scale='var', show=True, save='11_1. Dotplot_cd4Cluster_Curated_Markers2_leiden_1_0.pdf.pdf')
sc.pl.dotplot(adata_tcell, treg_genes_dict, 'leiden_1_0', dendrogram=False, use_raw=False, standard_scale='var', show=True, save='11_1. Dotplot_tregCluster_Curated_Markers2_leiden_1_0.pdf.pdf')
sc.pl.dotplot(adata_tcell, tfh_genes_dict, 'leiden_1_0', dendrogram=False, use_raw=False, standard_scale='var', show=True, save='11_1. Dotplot_tfhCluster_Curated_Markers2_leiden_1_0.pdf.pdf')

The dotplot visualization provides a compact way of showing per group, the fraction of cells expressing a gene (dot size) and the mean expression of the gene in those cell (color scale).

**<span style="font-size:16px;">Create a dictionary to map cluster to annotation label</span>**

In [ ]:
cluster2annotation = {
     '0': 'Proliferating T', 
     '1': 'CD8⁺ GZMK⁺ Tpex',
     '2': 'CD8⁺ GZMB⁺ early Tem', 
     '3': 'CD8⁺ MT⁺ Tstr',
     '4': 'CD8⁺ GZMB⁺ Tem',
     '5': 'CD8⁺ ISG⁺ T',
     '6': 'Proliferating T',
     '7': 'CD4⁺ IL21⁺ Tfh',
     '8':  'Undefined',
     '9':  'Undefined',
     '10': 'Undefined',
     '11': 'CD8⁺ GZMB⁺ Tem',
     '12': 'Undefined',
     '13': 'CD4⁺ ISG⁺ T',
     '14': 'Undefined',
     '15': 'CD4⁻CD8⁻ T', 
     '16': 'Undefined',
     '17': 'CD4⁻CD8⁻ T',
     '18': 'CD4⁻CD8⁻ T',
     '19': 'TNFRSF9⁻ Treg',
     '20': 'CD4⁻CD8⁻ T',
     '21': 'TNFRSF9⁺ Treg',
     '22': 'CD8⁺ ZNF683⁺ Trm',
     '23': 'CD8⁺ ZNF683⁺ Trm',
     '24': 'Undefined',
     '25': 'CD4⁺ Tcm',
     '26': 'CD8⁺ GZMB⁺ Tex',
     '27': 'CD8⁺ GZMB⁺ Tem', 
     '28': 'CD8⁺ GZMB⁺ Tem',
}
adata_tcell.obs['T_subtype'] = adata_tcell.obs['leiden_1_0'].map(cluster2annotation).astype('category')
adata_tcell.obs['T_subtype'] = adata_tcell.obs['T_subtype'].cat.reorder_categories(['CD8⁺ GZMB⁺ early Tem','CD8⁺ GZMB⁺ Tem', 'CD8⁺ GZMK⁺ Tpex', 'CD8⁺ GZMB⁺ Tex', 'CD8⁺ MT⁺ Tstr', 'CD8⁺ ZNF683⁺ Trm', 'CD8⁺ ISG⁺ T',
                                                                                    'CD4⁺ Tcm', 'CD4⁺ ISG⁺ T', 'CD4⁺ IL21⁺ Tfh', 'TNFRSF9⁺ Treg', 'TNFRSF9⁻ Treg',
                                                                                    'CD4⁻CD8⁻ T', 'Proliferating T','Undefined'])

In [ ]:
adata_tcell0 = adata_tcell[adata_tcell.obs['T_subtype'] != 'Undefined'].copy()

In [ ]:
scores = dc.pp.get_obsm(adata_tcell0, key='score_aucell')

sc.pl.matrixplot(
    adata=scores,
    var_names=scores.var_names,
    groupby='T_subtype',
    dendrogram=True,
    standard_scale='var',
    colorbar_title='Z-scaled scores',
    cmap='RdBu_r',
)

In [ ]:

target_subtypes = [
    'CD8⁺ GZMB⁺ early Tem',
    'CD8⁺ GZMB⁺ Tem',
    'CD8⁺ GZMK⁺ Tpex',
    'CD8⁺ GZMB⁺ Tex',
    'CD8⁺ MT⁺ Tstr',
    'CD8⁺ ZNF683⁺ Trm',
    'CD8⁺ ISG⁺ T'
]

cd8_scores = scores[scores.obs["T_subtype"].isin(target_subtypes)].copy()
cd8_scores.obs["T_subtype"] = cd8_scores.obs["T_subtype"].str.replace(r"^CD8⁺\s+", "", regex=True)

plt.figure(figsize=(8, 6))
sc.pl.violin(
    adata=cd8_scores,
    keys=['Exhaustion'],
    groupby='T_subtype',
    jitter=False,
    rotation=45,
    show=False
)

plt.gcf().set_size_inches(6, 6) 

plt.tight_layout()
plt.savefig("Results/11.TCell/11.violin_exhaustion_by_CD8T_subtype.png", dpi=300)

In [ ]:
import matplotlib.pyplot as plt
import distinctipy

n_types = adata_tcell0.obs['T_subtype'].nunique()
colors = distinctipy.get_colors(n_types)
colors_hex = [distinctipy.get_hex(color) for color in colors]

fig, ax = plt.subplots(figsize=(6,5)) 
ov.pl.embedding(
    adata=adata_tcell0,
    basis='X_umap',
    color=['T_subtype'],
    palette=colors_hex,
    show=False,
    add_outline=False,
    frameon='small',
    legend_fontoutline=2,
    ax=ax
)

plt.savefig('Results/11.TCell/11.TCell_subtype_UMAPplot.pdf', format='pdf', bbox_inches='tight')
plt.show()

### Compositional data visualization
Analyzing compositional data is not straightforward. scCODA provides some ways of visualizing the properties of a compositional dataset before analysis. We will showcase these functions on the data on pathogen infection of mice from *Haber et al. [2017]*.

In [ ]:
fig, ax = plt.subplots(figsize=(5, 5))
ov.pl.cellproportion(
    adata=adata_tcell0,
    celltype_clusters='T_subtype',
    groupby='EBV_status',
    legend=True,
    ax=ax,
)

legend = ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), fontsize=10)
xticks = ax.get_xticks()
ax.set_xticks(xticks)  
ax.set_xticklabels([str(i+1) for i in range(len(xticks))])
ax.set_xlabel('')
ax.tick_params(axis='x', rotation=90, labelsize=6.5) 
plt.tight_layout()
plt.savefig("Results/11.TCell/11.Stacked_Barplot_of_Myeloid_Composition.pdf", format='pdf', dpi=300, bbox_inches='tight')
plt.show()

#### Save AnnData object with automated celltype annotation

In [ ]:
adata_tcell0

In [ ]:
print(np.min(adata_tcell0.X), np.max(adata_tcell0.X))

In [ ]:
adata_tcell0.write_h5ad("Processed Data/scRNA_TCell.h5ad")


**<span style="font-size:16px;">Session information：</span>**

In [ ]:
import sys
import platform
import pkg_resources

# Get Python version information
python_version = sys.version
# Get operating system information
os_info = platform.platform()
# Get system architecture information
architecture = platform.architecture()[0]
# Get CPU information
cpu_info = platform.processor()
# Print Session information
print("Python version:", python_version)
print("Operating system:", os_info)
print("System architecture:", architecture)
print("CPU info:", cpu_info)

# Print imported packages and their versions
print("\nImported packages and their versions:")
for package in pkg_resources.working_set:
    print(package.key, package.version)